In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import KFold

from sklearn.svm import SVR

In [2]:
train = pd.read_csv('/kaggle/input/predictingbookratingsint20h/train.csv')
test = pd.read_csv('/kaggle/input/predictingbookratingsint20h/test.csv')
submission = pd.read_csv('/kaggle/input/predictingbookratingsint20h/submission.csv')

In [3]:
train.drop(columns = 'id', inplace = True)
test.drop(columns = 'id', inplace = True)

## Checks****

In [4]:
"""test[test.book_title.isin(train[train.book_title.isin(test.book_title)].book_title)].book_title.value_counts()

train[train.book_title.isin(train.book_title.unique()[train.book_title.value_counts()>1])]

tmp =  train.groupby(['book_title', 'book_authors']).book_rating.apply(lambda x: len(set(x)) == 1)
not_unique = tmp[tmp==False].reset_index().book_title
train.book_title.nunique()

plt.figure(figsize=(16,8))
sns.distplot(train['book_rating'])
plt.axvline(train['book_rating'].mean(), color='b')
plt.axvline(train['book_rating'].median(), color='g')

plt.xticks(fontsize=15, color='k')
plt.yticks(fontsize=15, color='k')
plt.xlabel('Average Rating', fontsize=20, color='k')
plt.show()"""

"test[test.book_title.isin(train[train.book_title.isin(test.book_title)].book_title)].book_title.value_counts()\n\ntrain[train.book_title.isin(train.book_title.unique()[train.book_title.value_counts()>1])]\n\ntmp =  train.groupby(['book_title', 'book_authors']).book_rating.apply(lambda x: len(set(x)) == 1)\nnot_unique = tmp[tmp==False].reset_index().book_title\ntrain.book_title.nunique()\n\nplt.figure(figsize=(16,8))\nsns.distplot(train['book_rating'])\nplt.axvline(train['book_rating'].mean(), color='b')\nplt.axvline(train['book_rating'].median(), color='g')\n\nplt.xticks(fontsize=15, color='k')\nplt.yticks(fontsize=15, color='k')\nplt.xlabel('Average Rating', fontsize=20, color='k')\nplt.show()"

## Preprocess

In [5]:
train['book_genre'] = train.book_genre.fillna('')
train["book_genre"] = train.book_genre.str.replace('|',',')
train['book_authors'] = train.book_authors.str.replace('|',',')

train["book_format"] = train["book_format"].fillna("no_format")
#fill nan with mean over format
train["book_pages"].loc[~train["book_pages"].isnull()] = (
    train["book_pages"].loc[~train["book_pages"].isnull()].apply(lambda x: int(x[:-5]))
)
train["book_pages"] = train["book_pages"].astype(np.float32)
train["book_pages"] = train["book_pages"].fillna(
    train.groupby("book_format")["book_pages"].transform("mean")
)
train["book_pages"] = train["book_pages"].fillna(0)
train["book_pages"] = train["book_pages"].astype(np.int32)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [6]:
train.isna().sum()

book_title             0
book_image_url       392
book_desc              0
book_genre             0
book_authors           0
book_format            0
book_pages             0
book_review_count      0
book_rating_count      0
book_rating            0
dtype: int64

In [7]:
all_formats = np.array(
    list(
        set(
            test["book_format"].unique().tolist()
            + train["book_format"].unique().tolist()
        )
    )
).astype(np.object)
all_generes = np.array(
    list(
        set(
            train.book_genre.str.split(",").explode().unique().tolist()
            + test.book_genre.str.split("|").explode().unique().tolist()
        )
    )
).astype(np.object)


In [8]:
to_replace = all_formats[np.isin(all_formats,all_generes)]
all_formats[np.isin(all_formats,all_generes)] = to_replace+" format"

In [9]:
generes_bin = MultiLabelBinarizer().fit([all_generes])
format_bin = MultiLabelBinarizer().fit([all_formats])

In [10]:
train = train.join(pd.DataFrame(generes_bin.transform(train['book_genre'].apply(lambda x: x.split(','))),
                          columns=generes_bin.classes_,
                          index=train.index))
train = train.join(pd.DataFrame(format_bin.transform(train['book_format'].apply(lambda x: [x])),
                          columns=format_bin.classes_,
                          index=train.index))
train.drop(columns = ['book_genre', 'book_format','book_image_url','book_title','book_desc', 'book_authors'],inplace = True)

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:995: UserWarning: unknown class(es) ['Audiobook', 'Diary', 'Fiction', 'Graphic Novels'] will be ignored
  .format(sorted(unknown, key=str)))


In [11]:
train

,book_pages,book_review_count,book_rating_count,book_rating,,10th Century,11th Century,12th Century,13th Century,14th Century,...,no_format,online fiction,online serial,paper,paperback,"paperback, Kindle eBook",revised edition,softcover,web,単行本
0,768,40197,668892,4.22,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,180,27,126,3.95,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,470,331,3626,3.79,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,106,28,371,3.85,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,110,250,4331,3.72,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,491,24661,165284,4.46,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39996,48,130,1556,4.08,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39997,416,3,159,4.20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39998,336,645,2955,3.62,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
train.isna().sum().sum()

0

In [13]:
train

,book_pages,book_review_count,book_rating_count,book_rating,,10th Century,11th Century,12th Century,13th Century,14th Century,...,no_format,online fiction,online serial,paper,paperback,"paperback, Kindle eBook",revised edition,softcover,web,単行本
0,768,40197,668892,4.22,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,180,27,126,3.95,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,470,331,3626,3.79,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,106,28,371,3.85,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,110,250,4331,3.72,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,491,24661,165284,4.46,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39996,48,130,1556,4.08,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39997,416,3,159,4.20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39998,336,645,2955,3.62,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
train = train[["book_review_count","book_rating_count","book_rating"]]

## Linear regression kek



In [44]:
cv = []
X,y = train.drop(['book_rating'], axis=1), train['book_rating']/5
factors = X.max()
X = X/factors
X = X.fillna(0)

In [45]:
from sklearn.linear_model import BayesianRidge

In [52]:
for train_index, test_index in KFold(n_splits=5, random_state=42, shuffle=True).split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model = BayesianRidge().fit(X_train,y_train)
    preds = model.predict(X_test)

    cv.append(mean_squared_error((preds*5).round(2), y_test*5)**0.5)
    print(model.__class__.__name__+": "+str(cv[-1]))
    print("Base: "+ str(mean_squared_error([y_test.median()*5]*len(y_test), y_test*5)))
    print()

AttributeError: 'BayesianRidge' object has no attribute '__name__'

4.01